In [1]:
#! pip install pandas_datareader

In [2]:
import pandas as pd
import numpy as np
from pandas_datareader import wb
import matplotlib.pyplot as plt


In [3]:
wb.search('Inflation')

,id,name,unit,source,sourceNote,sourceOrganization,topics
6873,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",,World Development Indicators,Inflation as measured by the consumer price in...,"b'International Monetary Fund, International F...",Economy & Growth ; Financial Sector
6875,FP.FPI.TOTL.ZG,"Inflation, food prices (annual %)",,WDI Database Archives,,b'',
6877,FP.WPI.TOTL.ZG,"Inflation, wholesale prices (annual %)",,WDI Database Archives,,b'',
10988,NY.GDP.DEFL.87.ZG,"Inflation, GDP deflator (annual %)",,WDI Database Archives,,b'',
10989,NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)",,World Development Indicators,Inflation as measured by the annual growth rat...,"b'World Bank national accounts data, and OECD ...",Economy & Growth
10990,NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)",,World Development Indicators,Inflation as measured by the annual growth rat...,b'World Bank staff estimates based on World Ba...,Financial Sector


In [4]:
GDP_Current_LCU = 'NY.GDP.MKTP.CN'
GDP_Constant_LCU = 'NY.GDP.MKTP.KN'
BroadMoney = 'FM.LBL.BMNY.CN'
Inflation_GDP_Deflactor = 'NY.GDP.DEFL.KD.ZG'
indicators = [GDP_Current_LCU, GDP_Constant_LCU, BroadMoney, Inflation_GDP_Deflactor]
cols = {'NY.GDP.MKTP.CN': 'GDP_Current','NY.GDP.MKTP.KN': 'GDP_Constant', 'FM.LBL.BMNY.CN': 'Broad_Money', 'NY.GDP.DEFL.KD.ZG': 'Inflation' }

In [6]:
for id in indicators:
    df = wb.download(indicator=id, country='ARG', start=1960, end=2023)
    df = df.reset_index()
    df.rename(columns=cols, inplace=True)
    archivo_csv = f'csv_creados\{id}.csv'
     
    df.to_csv(archivo_csv, index=False)
    print(f'Se ha guardado "{archivo_csv}"')

<string>:5: SyntaxWarning: invalid escape sequence '\{'
<>:5: SyntaxWarning: invalid escape sequence '\{'
<string>:5: SyntaxWarning: invalid escape sequence '\{'
<>:5: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Ariel\AppData\Local\Temp\ipykernel_9320\2704164009.py:5: SyntaxWarning: invalid escape sequence '\{'
  archivo_csv = f'csv_creados\{id}.csv'
C:\Users\Ariel\AppData\Local\Temp\ipykernel_9320\2704164009.py:5: SyntaxWarning: invalid escape sequence '\{'
  archivo_csv = f'csv_creados\{id}.csv'


ReadTimeout: HTTPSConnectionPool(host='api.worldbank.org', port=443): Read timed out. (read timeout=30)

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
archivos = ['csv_creados/NY.GDP.MKTP.CN.csv', 'csv_creados/NY.GDP.MKTP.KN.csv', 'csv_creados/FM.LBL.BMNY.CN.csv', 'csv_creados/NY.GDP.DEFL.KD.ZG.csv']
names = ['GDP_N', 'GDP_R', 'M', 'Inf']


for archivo, name in zip(archivos, names):
   
    df = pd.read_csv(archivo)
    df.drop(columns=['country'], inplace=True)
   
    globals()[f'df_{name}'] = df

In [ ]:
df_GDP_N.info()

In [ ]:
df = df_GDP_N.merge(df_GDP_R, on=['year'], how='inner').merge(df_M, on=['year'], how='inner')

In [ ]:
df['GDP_Constant'] = df['GDP_Constant'].astype(float)
df['year'] = pd.to_datetime(df['year'].astype(str) + '1231', format='%Y%m%d')
df = df.sort_values(by='year').reset_index(drop=True)

In [ ]:
df.head(2)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

axs[0].plot(df['year'], df['GDP_Current'], label='GDP_Current', color='blue')
axs[0].set_xlabel('Year')
axs[0].set_ylabel('GDP Current (Log Scale)')
axs[0].set_title('Argentina GDP Current (LCU) 1960-2023')
axs[0].set_yscale('log')
axs[0].legend()

axs[1].plot(df['year'], df['GDP_Constant'], label='GDP_Constant', color='green')
axs[1].set_xlabel('Year')
axs[1].set_ylabel('GDP Constant')
axs[1].set_title('Argentina GDP Constant (LCU) 1960-2023')
axs[1].legend()


axs[2].plot(df['year'], df['Broad_Money'], label='Broad Money', color='red')
axs[2].set_xlabel('Year')
axs[2].set_ylabel('Broad Money (Log Scale)')
axs[2].set_title('Argentina Broad Money (LCU) 1960-2023')
axs[2].set_yscale('log')
axs[2].legend()


plt.tight_layout()


plt.show()

In [ ]:
df.to_csv('csv_creados/Data_Argentina_Absolute_Terms.csv', index=False)

In [ ]:
df_arc = df.copy()
for col in df.columns[1:]: 
    df_arc[col] = df[col].pct_change()

In [ ]:
df_arc.head(2)

In [ ]:
df_Inf['Inflation'] = df_Inf['Inflation'] / 100
df_Inf['year'] = pd.to_datetime(df_Inf['year'].astype(str) + '1231', format='%Y%m%d')
df_Inf = df_Inf.sort_values(by='year').reset_index(drop=True)


In [ ]:
df_arc = df_arc.merge(df_Inf, on=['year'], how='inner')

In [ ]:
df_arc.isnull().sum()

In [ ]:
df_arc = df_arc.dropna()
df_arc = df_arc.reset_index(drop=True)

In [ ]:
df_arc.head(2)

Ecuation of Exchange:

m v = p q

p q = gdp

v = gdp - m

* small letters for variables in anual rate of change

In [ ]:
df_arc['Velocity_Money'] = df_arc['GDP_Current'] - df_arc['Broad_Money']

In [ ]:
df_arc.to_csv('csv_creados/Data_Argentina_Relative_Terms.csv',index=False) 

In [ ]:
plt.figure(figsize=(8, 4))
plt.title('Velocity of Money 1960 - 2023')
plt.xlabel('year')
plt.ylabel('Velocity of Money')

plt.plot(df_arc['year'], df_arc['Velocity_Money'])

plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.title('Broad Money Growth and Inflation 1961 - 2023')
plt.xlabel('Año')
plt.ylabel('Anual Rate of Change')

# Graficar las líneas temporales para M0, M1, M2 y M3
plt.plot(df_arc['year'], df_arc['Broad_Money'], color='b', label='Broad Money Growth')
plt.plot(df_arc['year'], df_arc['Inflation'], color='r', label='Inflation Rate')


# Mostrar la leyenda
plt.legend()

# Mostrar la gráfica
plt.tight_layout()
plt.show()

In [ ]:
df_arc.head()